# Forces 🏋️

- [ForceGraph Top level API](https://github.com/vasturiano/force-graph#force-engine-d3-force-configuration)
  - Forcing Functions:
    - `d3Force(str, [fn])`
    - Default forces: {"link", "charge", "center"}
    - Example of
      ["collide" and "box"](https://vasturiano.github.io/force-graph/example/collision-detection/)([source](https://github.com/vasturiano/force-graph/blob/master/example/collision-detection/index.html))
    - https://www.d3indepth.com/force-layout/
      - forceCenter
      - forceCollide
        - [using mouse](https://observablehq.com/@d3/collision-detection/2?collection=@d3/d3-force)
      - forceLink
      - forceManyBody
      - forceRadial
      - forceX
      - forceY
    - [full on custom](https://technology.amis.nl/frontend/introduction-to-d3-force-for-simulation-and-animation/)
      - use handlebar templates?
  - [DAG](https://github.com/vasturiano/force-graph/blob/master/example/tree/index.html)

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import asyncio
import random

import ipywidgets as W
import traitlets as T

import ipyforcegraph.forces as F
from ipyforcegraph.utils import wait_for_change

In [ ]:
with __import__("importnb").Notebook():
    import Behaviors as B
    import Utils as U

In [ ]:
if __name__ == "__main__":
    fg, box = U.make_a_demo()
    B.add_graph_data(fg, box)
    graph_data = box.behaviors["graph_data"]
    display(box)

## `GraphForcesBehavior`

If the built-in forces do not meet the needs of a particular application, these can be
customized in many ways. These are collected under the `GraphForces` behavior, which has
both top-level parameters, common to the overall simulation engine, as well is
individual named forces inside of it, which are themselves highly configurable.

### Warmup and Cooldown

These `warmup_ticks` and `cooldown_ticks` parameters control how the simulation starts
up and how long it is allowed to run.

By default, `cooldown_ticks` is set to `-1`, meaning the simulation will be allowed to
run as long as it needs before reaching a steady state.

When `cooldown_ticks` is set to `0`, the simulation won't run at all, useful for
pre-calculated positions.

`warmup_ticks` controls how much is simulated off-screen, before any force-driven
animation begins.

In [ ]:
def add_force_ticks(fg, box):
    gf = U.ensure_graph_forces(fg, box)

    warmup = W.IntSlider(description="warmup", min=0, max=100)
    cooldown = W.IntSlider(description="cooldown", min=-1, max=100)

    T.link((gf, "warmup_ticks"), (warmup, "value"))
    T.link((gf, "cooldown_ticks"), (cooldown, "value"))

    ui_ticks = U.make_a_collapsible_picker(
        "graph: warmup & cooldown",
        {
            "spoon": [W.VBox([warmup, cooldown], layout=dict(width="100%"))],
        },
    )
    box.node_ui = {**box.node_ui, "graph_ticks": ui_ticks}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_force_ticks(fg, box)

### Alpha and Velocity

The parameters `alpha_min`, `alpha_decay` and `velocity_decay` fine-tune the natural
stopping state of the simulation.

In [ ]:
def add_alpha_velocity(fg, box):
    gf = U.ensure_graph_forces(fg, box)

    alpha_min = W.FloatSlider(
        description="alpha min", min=0, max=1, step=0.0001, readout_format=".4f"
    )
    alpha_decay = W.FloatSlider(
        description="alpha decay", min=0, max=1, step=0.0001, readout_format=".4f"
    )
    velocity_decay = W.FloatSlider(
        description="velocity decay", min=0, max=1, step=0.0001, readout_format=".4f"
    )

    T.link((gf, "alpha_min"), (alpha_min, "value"))
    T.link((gf, "alpha_decay"), (alpha_decay, "value"))
    T.link((gf, "velocity_decay"), (velocity_decay, "value"))

    ui_ticks = U.make_a_collapsible_picker(
        "graph: alpha & velocity",
        {
            "av": [
                W.VBox(
                    [alpha_min, alpha_decay, velocity_decay], layout=dict(width="100%")
                )
            ],
        },
    )
    box.node_ui = {**box.node_ui, "graph_alpha_velocity": ui_ticks}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_alpha_velocity(fg, box)

## Base Forces

By default the forcegraph uses the following forces:

- `link` - LinkForce
- `charge` - ManyBodyForce
- `force_center` - CenterForce

In [ ]:
def add_base_forces(fg, box):
    gf = U.ensure_graph_forces(fg, box)

    force_link = F.LinkForce()
    force_charge = F.ManyBodyForce()
    force_center = F.CenterForce()

    forces = {
        "link": force_link,
        "charge": force_charge,
        "center": force_center,
    }
    gf.forces = {**gf.forces, **forces}

    # Link UI
    link_ui = W.FloatSlider(
        0.2, min=0, max=1.9, description="strength", layout={"flex": "1"}
    )
    T.dlink((link_ui, "value"), (force_link, "strength"), str)

    # Charge UI
    charge_ui = W.FloatSlider(
        description="charge", min=-50, value=-30, max=10, layout={"flex": "1"}
    )
    T.dlink((charge_ui, "value"), (force_charge, "strength"), str)

    # Center UI
    center_sliders = dict(
        x=W.FloatSlider(description="X", min=-200, max=200, layout={"flex": "1"}),
        y=W.FloatSlider(description="Y", min=-200, max=200, layout={"flex": "1"}),
        z=W.FloatSlider(description="Z", min=-200, max=200, layout={"flex": "1"}),
    )

    for key in ["x", "y", "z"]:
        T.link((center_sliders[key], "value"), (force_center, key))
    center_ui = W.VBox(list(center_sliders.values()), layout={"flex": "1"})

    children = {
        "link": [link_ui],
        "charge": [charge_ui],
        "center": [center_ui],
    }
    force_ui, children = U.ensure_graph_force_ui(fg, box, children)
    return fg, box

In [ ]:
if __name__ == "__main__":
    fg, box = add_base_forces(fg, box)

## `CollisionForce`

The collision force treats nodes as circles with a given `radius`, rather than points
and prevents nodes from overlapping.

In [ ]:
def add_collide_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force_collide = F.CollisionForce(radius="4")
    gf.forces = {**gf.forces, "collide": force_collide}

    radius_slider = W.FloatSlider(
        description="Radius",
        min=0,
        max=100,
        value=int(force_collide.radius),
        layout={"flex": "1"},
    )
    T.dlink((radius_slider, "value"), (force_collide, "radius"), str)

    children = {
        "collide": [radius_slider],
    }
    force_ui, children = U.ensure_graph_force_ui(fg, box, children)

    return fg, box

In [ ]:
if __name__ == "__main__":
    add_collide_force(fg, box)

## `RadialForce`

The radial positioning force create a force towards a circle of the specified radius
centered at (x, y).

In [ ]:
def add_radial_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force_radial = F.RadialForce(radius="100", strength="0")
    forces = {
        "radial": force_radial,
    }
    gf.forces = {**gf.forces, **forces}

    x_slider = W.FloatSlider(
        description="X", min=-100, value=0, max=100, layout={"flex": "1"}
    )
    y_slider = W.FloatSlider(
        description="Y", min=-100, value=0, max=100, layout={"flex": "1"}
    )
    radius_slider = W.FloatSlider(
        description="Radius", min=0, value=10, max=200, layout={"flex": "1"}
    )
    strength_slider = W.FloatSlider(
        description="Strength", min=0, value=0, max=5, layout={"flex": "1"}
    )
    T.dlink((radius_slider, "value"), (force_radial, "radius"), str)
    T.dlink((strength_slider, "value"), (force_radial, "strength"), str)
    T.dlink((x_slider, "value"), (force_radial, "x"))
    T.dlink((y_slider, "value"), (force_radial, "y"))

    children = {
        "radial": [
            W.VBox(
                [
                    x_slider,
                    y_slider,
                    radius_slider,
                    strength_slider,
                ],
                layout={"flex": "1"},
            )
        ],
    }
    force_ui, children = U.ensure_graph_force_ui(fg, box, children)

    return fg, box


if __name__ == "__main__":
    add_radial_force(fg, box)

In [ ]:
all_forces = [
    add_force_ticks,
    add_alpha_velocity,
    add_base_forces,
    add_collide_force,
    add_radial_force,
]